Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
# Intial values
batch_size = 128
hidden_nodes = 1024
num_steps = 10001
train_subset = 200
# uncomment below for problem 3
#train_dataset = train_dataset[:train_subset].reshape(train_subset, 784)
#train_labels = train_labels[:train_subset]

Try out Logistic Regression first

In [14]:
# Logistic Regression
graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  # Create a place holder for all tensor
  lamda = tf.placeholder(tf.float32)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  # when calculate the loss add L2 loss of all tensor's weight
  loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))  + lamda * tf.nn.l2_loss(weights)
  #print("Lost is ",loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, lamda : 0.001}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
      valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.413137
Minibatch accuracy: 6.2%
Validation accuracy: 8.6%
Minibatch loss at step 1000: 1.930875
Minibatch accuracy: 77.3%
Validation accuracy: 78.5%
Minibatch loss at step 2000: 0.849375
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 0.804311
Minibatch accuracy: 83.6%
Validation accuracy: 81.7%
Minibatch loss at step 4000: 0.651060
Minibatch accuracy: 83.6%
Validation accuracy: 82.3%
Minibatch loss at step 5000: 0.595934
Minibatch accuracy: 85.9%
Validation accuracy: 81.5%
Minibatch loss at step 6000: 0.789877
Minibatch accuracy: 75.8%
Validation accuracy: 81.5%
Minibatch loss at step 7000: 0.697029
Minibatch accuracy: 78.9%
Validation accuracy: 82.3%
Minibatch loss at step 8000: 1.040539
Minibatch accuracy: 73.4%
Validation accuracy: 81.6%
Minibatch loss at step 9000: 0.764133
Minibatch accuracy: 82.8%
Validation accuracy: 82.2%
Minibatch loss at step 10000: 0.786199
Minibatch accuracy: 76.6%
Validation accura

Now trying out the Neural Network

In [15]:
# Neural Network with 1 layer of Relu
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    lamda = tf.placeholder(tf.float32)
    
    # add weight and bias for NN
    weight1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    bias1 = tf.Variable(tf.zeros([hidden_nodes]))
    weight2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    bias2 = tf.Variable(tf.zeros([num_labels]))
    
    # Calculate relu
    logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weight1) + bias1), weight2) + bias2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
            + lamda * (tf.nn.l2_loss(weight1) +  tf.nn.l2_loss(weight2))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weight1) + bias1), weight2) + bias2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weight1) + bias1), weight2) + bias2)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, lamda : 0.002}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1017.870056
Minibatch accuracy: 11.7%
Validation accuracy: 34.4%
Minibatch loss at step 500: 231.646820
Minibatch accuracy: 84.4%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 83.682076
Minibatch accuracy: 79.7%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 30.561722
Minibatch accuracy: 93.0%
Validation accuracy: 85.6%
Minibatch loss at step 2000: 11.524429
Minibatch accuracy: 92.2%
Validation accuracy: 86.4%
Minibatch loss at step 2500: 4.633270
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 3000: 2.116631
Minibatch accuracy: 84.4%
Validation accuracy: 86.4%
Minibatch loss at step 3500: 1.180107
Minibatch accuracy: 85.2%
Validation accuracy: 86.8%
Minibatch loss at step 4000: 0.705485
Minibatch accuracy: 90.6%
Validation accuracy: 86.7%
Minibatch loss at step 4500: 0.555942
Minibatch accuracy: 88.3%
Validation accuracy: 86.5%
Minibatch loss at step 5000: 0.545340
Minibatch accuracy: 86.7%
Validation

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Neural Network with 1 layer of Relu
---------
---

In [12]:
batch_size = 10
# Neural Network with 1 layer of Relu
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    lamda = tf.placeholder(tf.float32)
    
    # add weight and bias for NN
    weight1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    bias1 = tf.Variable(tf.zeros([hidden_nodes]))
    weight2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    bias2 = tf.Variable(tf.zeros([num_labels]))
    
    # Calculate relu
    logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weight1) + bias1), weight2) + bias2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + lamda * tf.nn.l2_loss(weight1) + lamda * tf.nn.l2_loss(weight2)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weight1) + bias1), weight2) + bias2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weight1) + bias1), weight2) + bias2)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, lamda : 0.002}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 902.393677
Minibatch accuracy: 10.0%
Validation accuracy: 18.7%
Minibatch loss at step 1000: 8239.452148
Minibatch accuracy: 30.0%
Validation accuracy: 57.0%
Minibatch loss at step 2000: 1001.295166
Minibatch accuracy: 30.0%
Validation accuracy: 30.6%
Minibatch loss at step 3000: 134.874191
Minibatch accuracy: 10.0%
Validation accuracy: 24.7%
Minibatch loss at step 4000: 20.244190
Minibatch accuracy: 10.0%
Validation accuracy: 25.0%
Minibatch loss at step 5000: 4.298836
Minibatch accuracy: 40.0%
Validation accuracy: 26.5%
Minibatch loss at step 6000: 2.686178
Minibatch accuracy: 0.0%
Validation accuracy: 28.0%
Minibatch loss at step 7000: 1.829907
Minibatch accuracy: 30.0%
Validation accuracy: 27.9%
Minibatch loss at step 8000: 2.323624
Minibatch accuracy: 0.0%
Validation accuracy: 28.0%
Minibatch loss at step 9000: 1.998835
Minibatch accuracy: 30.0%
Validation accuracy: 24.1%
Minibatch loss at step 10000: 2.330142
Minibatch accuracy: 0.0%
Validati

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [16]:
batch_size = 128

# Neural Network with 1 layer of Relu
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    lamda = tf.placeholder(tf.float32)
    
    # add weight and bias for NN
    weight1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    bias1 = tf.Variable(tf.zeros([hidden_nodes]))
    weight2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    bias2 = tf.Variable(tf.zeros([num_labels]))
    
    # Calculate relu
    training_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weight1) + bias1)
    dropout_layer = tf.nn.dropout(training_layer, 0.5)
    logits = tf.matmul(dropout_layer, weight2) + bias2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
            + lamda * (tf.nn.l2_loss(weight1) + tf.nn.l2_loss(weight2))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    #valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weight1) + bias1), weight2) + bias2)
    #test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weight1) + bias1), weight2) + bias2)
    layer_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weight1) + bias1)
    valid_prediction = tf.nn.softmax(tf.matmul(layer_valid, weight2) + bias2)
    layer_test = tf.nn.relu(tf.matmul(tf_test_dataset, weight1) + bias1)
    test_prediction = tf.nn.softmax(tf.matmul(layer_test, weight2) + bias2)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, lamda : 0.002}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1203.977539
Minibatch accuracy: 7.0%
Validation accuracy: 19.8%
Minibatch loss at step 1000: 84.817413
Minibatch accuracy: 76.6%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 11.582397
Minibatch accuracy: 88.3%
Validation accuracy: 85.2%
Minibatch loss at step 3000: 2.172864
Minibatch accuracy: 85.9%
Validation accuracy: 85.4%
Minibatch loss at step 4000: 0.777189
Minibatch accuracy: 89.1%
Validation accuracy: 85.8%
Minibatch loss at step 5000: 0.622823
Minibatch accuracy: 88.3%
Validation accuracy: 86.3%
Minibatch loss at step 6000: 0.698825
Minibatch accuracy: 80.5%
Validation accuracy: 85.8%
Minibatch loss at step 7000: 0.713037
Minibatch accuracy: 81.2%
Validation accuracy: 86.1%
Minibatch loss at step 8000: 0.835741
Minibatch accuracy: 76.6%
Validation accuracy: 85.9%
Minibatch loss at step 9000: 0.724241
Minibatch accuracy: 82.0%
Validation accuracy: 86.0%
Minibatch loss at step 10000: 0.639622
Minibatch accuracy: 85.2%
Validation a

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [7]:
hidden_nodes1 = 512
hidden_nodes2 = 256
num_steps = 20001

# Neural Network with 1 layer of Relu
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # lamda = tf.placeholder(tf.float32)
    global_step = tf.Variable(0)
    lamda = tf.placeholder(tf.float32)
    
    # input layer to 1st layer connection 784 
    weight1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=0.1))
    bias1 = tf.Variable(tf.zeros([hidden_nodes]))
    
    # 1st layer to 2nd layer connection
    weight2 = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes1], stddev=0.1))
    bias2 = tf.Variable(tf.zeros([hidden_nodes1]))
    
    # 2nd layer to 3rd
    weight3 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2], stddev=0.1))
    bias3 = tf.Variable(tf.zeros([hidden_nodes2]))
    
    # 4th Layer
    weight4 = tf.Variable(tf.truncated_normal([hidden_nodes2, num_labels], stddev=0.1))
    bias4 = tf.Variable(tf.zeros([num_labels]))
    
    # Calculate relu for 1st Layer
    training_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weight1) + bias1)
    dropout_layer = tf.nn.dropout(training_layer, 0.5)
    
    # Calculate relu for 2nd Layer
    training_layer1 = tf.nn.relu(tf.matmul(dropout_layer, weight2) + bias2)
    dropout_layer1 = tf.nn.dropout(training_layer1, 0.5)
    
    # Calculate relu for 3rd Layer
    training_layer2 = tf.nn.relu(tf.matmul(dropout_layer1, weight3) + bias3)
    #training_layer2 = tf.nn.relu(tf.matmul(training_layer1, weight3) + bias3)
    dropout_layer2 = tf.nn.dropout(training_layer2, 0.5)
    
    # Calculate Logits and Loss
    logits = tf.matmul(dropout_layer2, weight4) + bias4
    #logits = tf.matmul(training_layer2, weight4) + bias4
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
        lamda * (tf.nn.l2_loss(weight1) + tf.nn.l2_loss(weight2) + tf.nn.l2_loss(weight3) + tf.nn.l2_loss(weight4))
    
    # Learning rate
    learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.80, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    layer_valid1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weight1) + bias1)
    layer_valid2 = tf.nn.relu(tf.matmul(layer_valid1, weight2) + bias2)
    layer_valid3 = tf.nn.relu(tf.matmul(layer_valid2, weight3) + bias3)
    valid_prediction = tf.nn.softmax(tf.matmul(layer_valid3, weight4) + bias4)
    
    layer_test1 = tf.nn.relu(tf.matmul(tf_test_dataset, weight1) + bias1)
    layer_test2 = tf.nn.relu(tf.matmul(layer_test1, weight2) + bias2)
    layer_test3 = tf.nn.relu(tf.matmul(layer_test2, weight3) + bias3)
    test_prediction = tf.nn.softmax(tf.matmul(layer_test3, weight4) + bias4)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, lamda : 0.002}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 24.156113
Minibatch accuracy: 14.8%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%


KeyboardInterrupt: 

In [19]:
# Test
# Neural Network with 1 layer of Relu
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    lamda = tf.placeholder(tf.float32)
    global_step = tf.Variable(0)
    
    # add weight and bias for NN
    weight1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    bias1 = tf.Variable(tf.zeros([hidden_nodes]))
    weight2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    bias2 = tf.Variable(tf.zeros([num_labels]))
    
    # Calculate relu
    training_layer = tf.nn.relu(tf.matmul(tf_train_dataset, weight1) + bias1)
    dropout_layer = tf.nn.dropout(training_layer, 0.5)
    logits = tf.matmul(dropout_layer, weight2) + bias2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
            + lamda * (tf.nn.l2_loss(weight1) + tf.nn.l2_loss(weight2))
    
    learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.80, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weight1) + bias1), weight2) + bias2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weight1) + bias1), weight2) + bias2)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, lamda : 0.002}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1136.528564
Minibatch accuracy: 14.8%
Validation accuracy: 22.5%
Minibatch loss at step 1000: 84.757805
Minibatch accuracy: 78.9%
Validation accuracy: 82.6%
Minibatch loss at step 2000: 11.546622
Minibatch accuracy: 89.8%
Validation accuracy: 85.5%
Minibatch loss at step 3000: 2.234370
Minibatch accuracy: 84.4%
Validation accuracy: 85.5%
Minibatch loss at step 4000: 0.790273
Minibatch accuracy: 86.7%
Validation accuracy: 86.0%
Minibatch loss at step 5000: 0.601636
Minibatch accuracy: 88.3%
Validation accuracy: 86.2%
Minibatch loss at step 6000: 0.698264
Minibatch accuracy: 82.8%
Validation accuracy: 86.5%
Minibatch loss at step 7000: 0.688494
Minibatch accuracy: 83.6%
Validation accuracy: 86.3%
Minibatch loss at step 8000: 0.834793
Minibatch accuracy: 78.1%
Validation accuracy: 86.0%
Minibatch loss at step 9000: 0.710646
Minibatch accuracy: 83.6%
Validation accuracy: 86.3%
Minibatch loss at step 10000: 0.712876
Minibatch accuracy: 82.0%
Validation 